In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from matplotlib import pyplot as plt

In [81]:
class KleibersLawDataset(Dataset):
    def __init__(self, path):
        data = np.loadtxt(path,delimiter=",")
        self.inputs = torch.from_numpy(np.log(data[:-1, :]))
        self.outputs = torch.from_numpy(np.log(data[-1:, :]))

    def __len__(self):
        return (len(self.inputs))

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

    def scatter_plot(self):
        x = self.inputs.numpy()
        y = self.outputs.numpy()
        plt.scatter(x,y)
        plt.grid()
        plt.show()

In [82]:
# load the dataset
training_data = KleibersLawDataset("./datasets/kleibers_law_data.csv")

# Show a plot of the raw data
# training_data.scatter_plot()

# Create a DataLoader
dataloader = DataLoader(training_data)

In [ ]:
# Define a linear model with torch.nn.Module
class LinearModel(torch.nn.Module):
    def __init__(self, input_dim):
        super(LinearModel, self).__init__()
        self.linear = torch.nn.Linear(input_dim, 1)

    def forward(self, x):
        return self.linear(x)

In [75]:
# instatiate the model based on the size of the input data
dataloader.dataset.inputs.shape

torch.Size([1498])